In [ ]:
#!/usr/bin/python3

import os
import numpy as np
import matplotlib.pyplot as plt
from srim import TRIM, Ion, Layer, Target
# from srim.output import Results, Range
import pandas as pd
from datetime import datetime

timeNow = datetime.now()
print("Today's date:", timeNow)
currentTime = timeNow.strftime("%Y%m%d-%H%M%S")

# user options
UsePreviousResults = 0
ResultsFolder0 = "results_20230925-130159_V52143Z"
# implantation energies in the folder
# Energy_eV = 1.e3 * np.arange(125, 400, 50)
# Energy_eV = np.array([100.,160.,220.,280.,330.,360.,390.])*1.e3
Energy_eV = np.array([100.,160.,220.,280.,310.,340.])*1.e3
# Energy_eV = np.array([100.,160.,220.,280.,330.])*1.e3
# Energy_eV = np.array([100.,160.,220.,280.,310.,340.])*1.e3
Doses = 2.e14 * np.ones(len(Energy_eV))

ResultsFolder = []
for E_eV in Energy_eV:
    ResultsFolder.append(ResultsFolder0+"/"+str(int(E_eV))+"_eV")

DataType = 2 # 1 for RANGE and 2 for VACANCY
PlotResults = 1
# deltaX_interp = 20 #nm

# specify the directory of SRIM.exe
# for Windows users the path will be of the form "C://..."
srim_executable_directory = "C://Users//micha//Desktop//SRIM-2013"

# sample details
sample = 'QSR212B'
WithMetals = 0
angle_ions_user = 7

d_0 = 0.0 # nm
d_Au = 250.0 # nm
d_Pt = 150.0 # nm
d_ingaas = 200.0 # nm
d_quatP = 20.0 # nm
d_inp = 2100.0 # nm
d_extra = 500.0 # nm

x_origin = d_0
x_Au = x_origin+d_Au
x_Pt = x_Au+d_Pt
if WithMetals:
    x_ingaas = x_Pt+d_ingaas
else:
    x_ingaas = x_origin+d_ingaas
x_quatP = x_ingaas+d_quatP
x_inp = x_quatP+d_inp
x_final = x_inp + d_extra
    
if not UsePreviousResults:
    
    # desired implantation energies (eV)
    Energy_eV = np.array([100.,140.,170.,200.,220.,240.,260.])*1.e3
    # Energy_eV = np.array([100.,150.,200.,240.,290.,320.,350.])*1.e3
    # Energy_eV = np.array([100.,150.,200.,250.,300.,330.,360.])*1.e3
    # Energy_eV = np.array([100.,160.,220.,280.,310.,340.])*1.e3
    Doses = 2.e14 * np.ones(len(Energy_eV))
    
    # number of simulated ions
    N_Ions = 1000
    
    # mass of the implanted ion 8Li (amu)
    ImplantIonMass = 1.008
    
    # use an empty list to hold each layer
    layers = []
    
    if WithMetals == 1:
        Gold = Layer(
            {"Au": {"stoich": 1.0}},
            density=19.311,
            width=d_Au*10,
            name="Gold",
        )
        layers.append(Gold)
        
        Pt = Layer(
            {"Pt": {"stoich": 1.0}},
            density=21.45,
            width=d_Pt*10,
            name="Pt",
        )
        layers.append(Pt)
    
    InGaAs = Layer(
        {"In": {"stoich": 1.0}, "Ga": {"stoich": 1.0}, "As": {"stoich": 1.0},},
        density=5.5,
        width=d_ingaas*10,
        name="InGaAs",
    )
    layers.append(InGaAs)
    
    InGaAsP = Layer(
        {"In": {"stoich": 1.0}, "Ga": {"stoich": 1.0}, "As": {"stoich": 1.0}, "P": {"stoich": 1.0},},
        density=5.065,
        width=d_quatP*10,
        name="InGaAsP",
    )
    layers.append(InGaAsP)
    
    InP_P = Layer(
        {"In": {"stoich": 1.0}, "P": {"stoich": 1.0},},
        density=4.81,
        width=d_inp*10,
        name="InP_P",
    )
    layers.append(InP_P)
    
    InP_P = Layer(
        {"In": {"stoich": 1.0}, "P": {"stoich": 1.0},},
        density=4.81,
        width=d_extra*10,
        name="InP_P",
    )
    layers.append(InP_P)
    
    # construct the mulitlayer target using the layers list
    target = Target(layers)
    
    count = 0
    output_directory = []
    # loop over implantation energies
    for E_eV in Energy_eV:
        # Construct the implanted 8Li ion
        ion = Ion("H", energy=E_eV, mass=ImplantIonMass)
        # Initialize a TRIM calculation with the ion & target
        trim = TRIM(
            target,
            ion,
            calculation=1,
            number_ions=N_Ions,
            description="%.0f eV 8H implanted in heterostructure" % E_eV,
            # reminders="",
            # autosave=1000,
            plot_mode=5,
            # plot_xmin=0,
            # plot_xmax=0,
            ranges=True,
            backscattered=True,
            transmit=True,
            sputtered=True,
            collisions=True,
            angle_ions=angle_ions_user,
        )
    
        # run the TRIM calculation
        # - this may take a few seconds to start (especially using wine)
        # - if all went well, a TRIM window will pop up!
        results = trim.run(srim_executable_directory)
        output_directory.append("results_"+sample+'_'+str(currentTime)+"/%.0f_eV" % E_eV)
        # move all the results for each implantation energy to a new directory
        os.makedirs(output_directory[count], exist_ok=True)
        TRIM.copy_output_files(srim_executable_directory, output_directory[count])
        count += 1
        ResultsFolder = output_directory

if PlotResults:
    Data = []
    for count_plot in range(0,len(Energy_eV)):
        filename = []
        folderName = ResultsFolder[count_plot]
        if DataType == 1:
            filename.append(folderName+'/RANGE.txt')
            MultiplyFactor = 1;
            header = ['Depth X (nm)','H+ Range (#/cm3)']
        else:
            if DataType == 2:
                filename.append(folderName+'/VACANCY.txt')
                MultiplyFactor = 1e8;
                header = ['Depth X (nm)','Produced vacancies (#/cm3)']

        print(filename[0])
        file = open(filename[0],'r')
        data = file.readlines()
        file.close()
        
        LinesWithData = 0
        for line in range(0, len(data)-1):
            try:
                tmp = int(data[line][0])
                LinesWithData += 1
                if LinesWithData == 1:
                    DataStartLine = line              
            except:
                pass
        DataEndLine = DataStartLine + LinesWithData - 1

        dataStr = []
        for line in range(DataStartLine, DataEndLine): #using only columns with data
            buffer = data[line].strip().split(' ') # remove empty values
            buffer = [x for x in buffer if str(x) != '']
            dataStr.append(buffer)

        dataStr = np.char.replace(dataStr, ',', '.')
        dataFloat = np.array(dataStr, dtype=np.float32) #converts to float numpy array
        dataManipulated = dataFloat
        
        dataManipulated[:,0] /= 10 #transform A in nm
        dataManipulated[:,1] = (dataFloat[:,1]+dataFloat[:,2])*MultiplyFactor #
        
        # number_x_range = int((x_inp-x_origin)/deltaX_interp)
        # new_x_range_data = np.linspace(deltaX_interp,x_inp,number_x_range)
        # new_y1_data = np.interp(new_x_range_data,dataFloat[:,0],dataFloat[:,1]*MultiplyFactor
        # new_y2_data = np.interp(new_x_range_data,dataFloat[:,0],dataFloat[:,2]*MultiplyFactor
        # dataFloatInterp = np.transpose(np.vstack((new_x_range_data,new_y1_data+new_y2_data)))
        dataSorted = {}
        for j in range(0,len(header)):
            # dataSorted[header[j]]=dataFloatInterp[:,j]
            dataSorted[header[j]]=dataManipulated[:,j]
            
        Data.append({'fileName':filename[0], 'data':pd.DataFrame(dataSorted), 'header':header})
        
    # In[2]:
    if DataType == 1:
        header = ['Depth X (nm)','H+ Range (#/cm3)']
    else:
        if DataType == 2:
            header = ['Depth X (nm)','Produced vacancies (#/cm3)']
            
    plt.clf()
    plt.xlabel(header[0])
    plt.ylabel(header[1])
    if DataType == 1:
        plt.title("H+ Range")
    else:
        if DataType == 2:
            plt.title("Vacancy Range")
    height = 0
    data_total = [0]*len(Data[0]['data'])
    for i in range(len(Data)):
        data2plot=np.asarray(Data[i]['data'])
        x_data2plot = data2plot[:,0]
        y_data2plot = np.multiply(data2plot[:,1],Doses[i])
        plt.plot(x_data2plot,y_data2plot,label = 'E = %d keV'%int(Energy_eV[i]/1000))
        data_total += np.multiply(data2plot[:,1],Doses[i])
    
    height = max(data_total)
    plt.plot(x_data2plot,data_total,label = 'Total')
    plt.plot([x_origin,x_origin],[0,height], linestyle = 'dotted', color="black")
    if WithMetals:
        plt.plot([x_Pt,x_Pt],[0,height], linestyle = 'dotted', color="black")
    plt.plot([x_ingaas,x_ingaas],[0,height], linestyle = 'dotted', color="black")
    plt.plot([x_quatP,x_quatP],[0,height], linestyle = 'dotted', color="black")
    plt.plot([x_inp,x_inp],[0,height], linestyle = 'dotted', color="black")
    
    # plt.yscale('log')
    plt.ylim([0,5e19])
    plt.legend(loc=(1.04, 0.5))
    plt.show()
    plt.tight_layout()
    
    if not UsePreviousResults:
        np.savetxt("results_"+sample+'_'+str(currentTime)+'/total.txt', np.column_stack([x_data2plot,y_data2plot]), fmt='%s', delimiter='\t')